In [27]:
import nltk
import random
import os
from sklearn import svm
import json
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import  PorterStemmer
import random
import pickle
import numpy as np

In [7]:
label_map={'Aspect1':1 ,'Aspect2':2 ,'Aspect3':3 ,'Aspect4':4 ,'Aspect5':5,'Aspect6':6 ,'Aspect7':7}

In [13]:
def load_dataset():  
    dataset=[]
    for root, dirs, files in os.walk('../Dataset'):
        for directory in dirs:
            for root_f, dirs_f, files_f in os.walk(os.path.join(root,directory)):
                for each_file in files_f:
                    with open(os.path.join(root,directory)+'/'+each_file) as f:
                        data_temp=json.load(f)    
                    dataset+=[(sentence,label_map[directory]) for sentence in data_temp]
    return dataset

In [10]:
def to_unigram(data):
    dataset=[]
    dataset+=[([items for items in sentence.split()],polarity) for sentence,polarity in data]    
    return dataset    

In [11]:
def get_words(x):    #returns all the words in the  data set
    vocab = []
    for sentence,polarity in x:  
        vocab.extend(sentence.split())
    return vocab

In [30]:
#execution
labeled_data=load_dataset()    #loading labeled sentences
random.shuffle(labeled_data)
with open('../NaiveBayes/vocab_unigram') as f:
    vocab=json.load(f)
#labeled_data=to_unigram(labeled_data)    #converting to unigram

In [25]:
#bigrams
print labeled_data[:10]

[([u'typic', u'day', u'work', u'learn', u'manag', u'co', u'worker', u'hardest', u'part', u'job', u'enjoy', u'part', u'job'], 1), ([u'although', u'abl', u'get', u'manag', u'skill', u'job', u'made', u'hard', u'balanc', u'work'], 1), ([u'great', u'experi', u'compani', u'past', u'year'], 6), ([u'work', u'learn', u'work'], 1), ([u'someon', u'look'], 7), ([u'realli', u'love', u'creat', u'beauti', u'softwar'], 7), ([u'meet', u'certain', u'criterion', u'time', u'manag', u'effici'], 4), ([u'job', u'learn', u'use', u'slip', u'leader'], 1), ([u'gener', u'construct', u'environ'], 6), ([u'work', u'life', u'balanc', u'good'], 4)]


In [18]:
print len(labeled_data)

201378


In [19]:
print len(vocab)
print vocab[:100]

6544
[u'foul', u'interchang', u'four', u'payoff', u'authorit', u'scold', u'lore', u'lord', u'digit', u'pigment', u'yellow', u'delv', u'fur', u'disturb', u'scholar', u'wooden', u'showca', u'habea', u'charter', u'erron', u'nigh', u'rustl', u'miller', u'histor', u'second', u'sooth', u'tether', u'dialogu', u'ruthless', u'thunder', u'specialist', u'aver', u'intellectu', u'hero', u'avert', u'accu', u'herd', u'gamut', u'china', u'cult', u'deterior', u'militari', u'k', u'pastur', u'fumbl', u'diplomat', u'golden', u'brought', u'sterl', u'stern', u'unit', u'spoke', u'overshadow', u'music', u'telegraph', u'passport', u'strike', u'expiat', u'paperwork', u'relay', u'relax', u'relat', u'notic', u'hurt', u'glass', u'exc', u'holi', u'outskirt', u'midst', u'hold', u'accid', u'blade', u'conceptu', u'sweeter', u'etiquett', u'household', u'centuri', u'cautiou', u'caution', u'want', u'gargantuan', u'shuffl', u'unpaid', u'travel', u'classifi', u'hot', u'hop', u'turk', u'perspect', u'hoy', u'diagram', u'wron

In [22]:
def extract_features(dataset):       #features are bag of words. document is a list of words of a sentence 
    feature_vector=[]
    labels=[]
    for sentence,label in dataset:
        features = {}
        labels.append(label)
        for word in vocab:
            features[word] = 0
            if word in sentence:
                features[word] = 1
        feature_vector.append(features.values())        
    return {'feature':feature_vector,'labels':labels}        

In [33]:
#features=extract_features(labeled_data[:10])
vectorizer = CountVectorizer(min_df=1)
corpus=[sentence for sentence,pol in labeled_data]
features=vectorizer.fit_transform(corpus)
labels=[label for sentence,label in labeled_data]

In [34]:
#features['labels']

In [35]:
classifier=svm.SVC()

In [36]:
classifier.fit(features,labels)
with open('svc_unigram_model','w') as f:
    pickle.dump(classifier,f)

In [11]:
sen='work life is bore'
classifier.classify(extract_features(sen.split()))

'neg'

In [21]:
dist = classifier.prob_classify(extract_features(sen.split()))
for label in dist.samples():
    print("%s: %f" % (label, dist.prob(label)))

neg: 0.585660
sli_pos: 0.035267
sli_neg: 0.230902
neutral: 0.123239
pos: 0.024931


In [22]:
polarity=0
for label in dist.samples():
    print("%s: %f" % (label, dist.prob(label)))
    polarity=max(polarity,dist.prob(label))
print polarity

neg: 0.585660
sli_pos: 0.035267
sli_neg: 0.230902
neutral: 0.123239
pos: 0.024931
0.585660041903
